In [1]:
%matplotlib inline
#import numpy as np
import pandas as pd
#import geopandas as gpd
#import shapely.geometry as sg
#import matplotlib.pyplot as plt
import folium
from folium import plugins
import datetime
import time

kpit = pd.read_csv('./data/kpit_july_2018_part.csv')
flights = pd.read_csv('./data/flights_kpit_july2018.csv')
geofence = pd.read_csv('./data/geofence_events_kpit_july2018.csv')


dt_range_start = datetime.datetime(2018,7,5,0,0,0)
dt_range_end = datetime.datetime(2018,7,6,0,0,0)
time_interval = ((kpit.timestamp/1000 >= time.mktime(dt_range_start.timetuple())) &
                 (kpit.timestamp/1000 < time.mktime(dt_range_end.timetuple())))
# Verify with this formula 
# kpit[time_interval].timestamp.apply(lambda x: datetime.datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))
sel_usr = kpit.user_id==313
usr = kpit[sel_usr & time_interval]


In [7]:
#%% Animated track plot
features = [{
                "type": 'Feature',
                "geometry": {
                    "type": 'LineString',
                    "coordinates": list(zip(usr.longitude.tolist(), usr.latitude.tolist()))
                    },
                "properties": {
                    "times": usr.timestamp.tolist()
                    }
            }]
    
tsgeojson_data = {
        'type': 'FeatureCollection',
        'features': features
        }

tsgeojson_track = plugins.TimestampedGeoJson(tsgeojson_data, 
                                             period='PT1M', 
                                             add_last_point=True,
                                             loop=True,
                                             transition_time=50)
            
f2 = folium.Figure(height = 400)
m2 = folium.Map(prefer_canvas=True)
m2.add_tile_layer(tiles='Cartodb positron')
m2.add_child(tsgeojson_track)

# turn on layer control
m2.add_child(folium.map.LayerControl())#Set the zoom to the maximum possible
m2.fit_bounds(m1.get_bounds())
#Save the map to an HTML file
#m2.save("./html_map_output/animated_track_plot.html")
#Show map
m2.add_to(f2)

In [4]:
#%% Pickup Area
f1 = folium.Figure(height=500, width='100%')
m1 = folium.Map(prefer_canvas=True)
m1.add_tile_layer(tiles='Cartodb positron')
pickup_lat = [40.4946,40.4971,40.4971,40.4946,40.4946]
pickup_long = [-80.2550,-80.2550,-80.25585,-80.25585,-80.2550]
pickup_box = list(zip(pickup_lat, pickup_long))
pickup_area=folium.PolyLine(locations=pickup_box,weight=2,color = 'blue')
polyline=folium.PolyLine(locations=list(zip(usr.latitude, usr.longitude)),weight=2,color = 'red')
m1.add_child(pickup_area)
m1.add_child(polyline)
m1.add_child(folium.map.LayerControl())
m1.fit_bounds(m1.get_bounds())
#m1.save("./html_map_output/track_plot.html")
m1.add_to(f1)

In [16]:
#%%
# Check and see if driver is within pickup box boundary
# Lat: 40.4946<x<40.4971
# Long: -80.25585<x<-80.255

in_pickup_area = usr[(usr.latitude>40.4946)&
                     (usr.latitude<40.4971)&
                     (usr.longitude>-80.25585)&
                     (usr.longitude<-80.255)
                     ]

pickup_timestamp = in_pickup_area.loc[in_pickup_area.speed.idxmin()].timestamp
pickup_datetime = datetime.datetime.fromtimestamp(pickup_timestamp/1000)
pickup_dtstring = pickup_datetime.strftime('%Y-%m-%d %H:%M:%S')

# Time when entering airport
entry_timestamp = usr.loc[usr.timestamp.idxmin()].timestamp
entry_datetime = datetime.datetime.fromtimestamp(entry_timestamp/1000)
entry_dtstring = entry_datetime.strftime('%Y-%m-%d %H:%M:%S')

# Duration in minutes between entry and pick-up
in_airport_duration = round((pickup_datetime - entry_datetime).seconds / 60)

print('entry time: ' + entry_dtstring)
print('pick-up time: ' + pickup_dtstring)
print('time-to-pickup: ' + str(in_airport_duration) + ' mins')

entry time: 2018-07-05 19:57:07
pick-up time: 2018-07-05 21:29:00
time-to-pickup: 92 mins
